## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

In [2]:
import string
from nltk.corpus import stopwords
from string import punctuation

Этапы:   
    1. получить коллекцию документов   --- +
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку --- +
    3. получить матрицу терм-документ, написать функцию поиска по ней --- +
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово,
    значение - список файлов, в которых это слово встречается  --- +
    5. вывести кусочек индекса в виде таблицы --- +
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы --- +

In [14]:
main_dir = 'Friends/'

In [15]:
stops = stopwords.words('russian')

In [16]:
from collections import defaultdict

In [17]:
import re

def remove(text):
    new = re.sub(r'\ufeff', '', text)
    new = re.sub(r'\n', ' ', new)
    new = re.sub(r'www.tvsubtitles.net', '', new)
    new = re.sub(r'([\W]|\d)', ' ', new)
    new = re.sub('(\n|\t|\v|\r)', ' ', new)
   # new = re.sub(r'[ \t\v\r\n\f]', ' ', new)
    return new

In [18]:
all_ = defaultdict(str)

In [19]:
idx = defaultdict(str)

In [20]:
from pymystem3 import Mystem

mystem = Mystem()

Словарь, чтобы потом понять сколько серия в каких сезонах

In [21]:
seasons = defaultdict(list)

Средняя длина текста (приблизительно)

In [22]:
avr = []

In [23]:
i = 0
l = 0
mystem = Mystem()

for root, dirs, files in os.walk(main_dir):
    for name in files:
        if not '.DS_Store' in name:
            with open(os.path.join(root, name), 'r') as f:
                text = f.read() 
                a = mystem.lemmatize(remove(text))
                avr.append(len(a))
                all_[name[:-7]] = ''.join(a)
                if i == 165:
                    break
                idx[i] = name[:-7]
                seasons[l].append(name[:-7])
                i += 1
    l += 1

Пример текста

In [25]:
print(all_[name[:-7]][0:200])

боже мой   я знать  у моника быть ребенок   это можно считать за  что то новенький    бог ты мой   думать  поэтому чендлер сбегать   должно быть  она проходить тест совсем недавно  я мусор вчера вечер


In [26]:
print(all_[name[:-7]][-200:])

ребенок   у мы быть ребенок   фиби находить твой тест на беременность   я не делать тест на беременность   тогда кто делать   посмотреть на они   у они быть ребенок                                   



In [27]:
len(all_)

165

Tf-idf матрица

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [29]:
vect = TfidfVectorizer()
x = vect.fit_transform(list(all_.values())) 

In [30]:
data = pd.DataFrame(x.todense()).transpose()

In [31]:
data.columns = list(idx.values())

In [32]:
data.shape

(14113, 165)

In [33]:
data['Term'] = vect.get_feature_names()

In [34]:
data = data.set_index(['Term'])

В качестве частоты будем использовать сумму по строчке.

In [35]:
data['freq'] = data.sum(axis=1)

In [36]:
data.head()

,Friends - 1x01 - The One Where Monica Gets A Roommate,Friends - 1x02 - The One With The Sonogram At The End,Friends - 1x03 - The One With The Thumb,Friends - 1x04 - The One With George Stephanopoulos,Friends - 1x05 - The One With The East German Laundry Detergent,Friends - 1x06 - The One With The Butt,Friends - 1x07 - The One With The Blackout,Friends - 1x08 - The One Where Nana Dies Twice,Friends - 1x09 - The One Where Underdog Gets Away,Friends - 1x10 - The One With The Monkey,...,Friends - 7x16 - The One With The Truth About London,Friends - 7x17 - The One With The Cheap Wedding Dress,Friends - 7x18 - The One With Joey's Award,Friends - 7x19 - The One With Ross And Monica's Cousin,Friends - 7x20 - The One With Rachel's Big Kiss,Friends - 7x21 - The One With The Vows,Friends - 7x22 - The One With Chandler's Dad,Friends - 7x23 - The One With Chandler And Monica's Wedding (1),Friends - 7x24-25 - The One With Chandler And Monica's Wedding (2),freq
Term,,,,,,,,,,,,,,,,,,,,,
after,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025744
again,0.0,0.02558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025580
ahh,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024608
all,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068349
and,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076152


Функция поиска

In [119]:
def data_search(data, word, episode=None, non=False):

    if not isinstance(word, str):
        raise ValueError
    if word == '':
        return None

    word = word.lower()

    try:
        data.loc[word]
    except KeyError:
        raise ValueError('There is no such word')

    if episode is None:
        answer = set()
        if not non: 
            for index, item in enumerate(data.loc[word].drop('freq')):
                if item > 0:
                    answer.add(idx[index])
        else:
            for index, item in enumerate(data.loc[word].drop('freq')):
                if item == 0:
                    answer.add(idx[index])
        return answer

    if not isinstance(episode, str):
        raise ValueError

    return data.loc[word][episode]

In [120]:
data_search(data, 'after')

{'Friends - 7x11 - The One With All The Cheesecakes'}

In [149]:
data_search(data, 'and')

{'Friends - 5x24-25 - The One In Vegas (2)',
 "Friends - 7x08 - The One Where Chandler Doesn't Like Dogs"}

Обратный индекс

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [26]:
main_dir = '/your/path/with/Frinds/collection/'
files_list = []

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов


In [187]:
def space(text):
    new = re.sub('\(', '( ', text)
    new = re.sub('\)', ' )', new)
    return new

In [269]:
### постройте матрицу терм-документ
term_doc_matrix = []


### напишите функцию булева поиска по построенной матрице
def boolean_search(data, text):
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """
    words = []
    op = []
    oper = ['&', 'ИЛИ', 'НЕ', '(', ')']
    text = space(text)
    text = text.split(' ')

    for item in text:

        if item not in oper:
            if len(op) > 0 and op[-1] == 'НЕ':
                item = data_search(data, item.lower(), non=True)
                op.pop()
            else:
                item = data_search(data, item.lower())

            if len(words) == 0:
                words.append(item)

            elif len(op) > 0:
                if op[-1] == '&':
                    answ = words[-1] & item
                    words.pop()
                    words.append(answ)
                    op.pop()
                elif op[-1] == 'ИЛИ':
                    answ = words[-1] | item
                    words.pop()
                    words.append(answ)
                    op.pop()
                elif op[-1] == '(':
                    op.pop()
                    words.append(item)

        elif item == ')':

            if len(op) <= 0:
                raise ValueError('Enter correct string')

            if op[-1] == '(':
                if len(words) <= 0:
                    raise ValueError('Enter correct string')

            else:
                for i in op[::-1]:

                    if i == '(':
                        break
                    elif i == '&':
                        answ = words[-2] & words[-1]
                        words.pop()
                        words.pop()
                        words.append(answ)
                        op.pop()
                    elif i == 'ИЛИ':
                        answ = words[-2] | words[-1]
                        words.pop()
                        words.pop()
                        words.append(answ)
                        op.pop()
                if len(op) == 0:
                    break

        else:
            op.append(item)
    return words[-1]


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джоуи & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джоуи) & Росс',
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джоуи & (НЕ Росс)'
]

In [271]:
text = 'Моника & Фиби & Рэйчел & Чендлер & Джоуи & Росс'
boolean_search(data, text)

{'Friends - 1x04 - The One With George Stephanopoulos',
 'Friends - 1x05 - The One With The East German Laundry Detergent',
 "Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip",
 'Friends - 2x02 - The One With The Breast Milk',
 "Friends - 2x04 - The One With Phoebe's Husband",
 'Friends - 2x07 - The One Where Ross Finds Out',
 'Friends - 2x17 - The One Where Eddie Moves In',
 'Friends - 2x20 - The One Where Old Yeller Dies.NurlanB',
 'Friends - 3x01 - The One With The Princess Leia Fantasy',
 'Friends - 3x03 - The One With The Jam',
 "Friends - 3x14 - The One With Phoebe's Ex-Partner",
 "Friends - 4x03 - The One With The 'Cuffs",
 'Friends - 4x10 - The One With The Girl From Poughkeepsie',
 'Friends - 4x15 - The One With All The Rugby',
 'Friends - 4x17 - The One With The Free Porn',
 'Friends - 4x21 - The One With The Invitations',
 "Friends - 4x24-25 - The One With Ross's Wedding (2)",
 'Friends - 5x02 - The One With All The Kissing',
 'Friends - 5x07 - The One Where Ross M

<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [272]:
def inverted_index(vect) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """

    d = defaultdict(list)

    for word in vect.get_feature_names():
        d[word] = [i for i in data_search(data, word)]

    return d

In [273]:
d = inverted_index(vect)

С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


Таблица индекса

In [274]:
import collections

In [275]:
inv_ind = pd.DataFrame({'term': vect.get_feature_names(),
                        'docs': list(d.values()),
                        'freq': data['freq']
                       })

In [276]:
inv_ind['lenght'] = inv_ind.docs.map(lambda x: len(x))

In [277]:
inv_ind.head()

,docs,freq,term,lenght
Term,,,,
after,[Friends - 7x22 - The One With Chandler's Dad],0.025744,after,1
again,[Friends - 7x12 - The One Where They're Up All...,0.025580,again,1
ahh,[Friends - 4x04 - The One With The Ballroom Da...,0.024608,ahh,35
all,[Friends - 7x15 - The One With Joey's New Brai...,0.068349,all,3
and,[Friends - 6x16-17 - The One That Could Have B...,0.076152,and,17


Какое слово является самым частотным?

In [278]:
inv_ind.ix[inv_ind['freq'].idxmax()]

docs      [Friends - 2x04 - The One With Phoebe's Husban...
freq                                                 55.332
term                                                     ты
lenght                                                    2
Name: ты, dtype: object

Какое самым редким?

In [279]:
inv_ind.ix[inv_ind['freq'].idxmin()]

docs      [Friends - 1x16 - The One With Two Parts (1)]
freq                                          0.0156364
term                                                 би
lenght                                                1
Name: би, dtype: object

Какой набор слов есть во всех документах коллекции?

In [280]:
list(inv_ind.loc[inv_ind['lenght'] == 165].term)

['done',
 'oo',
 'агрегат',
 'базар',
 'бублик',
 'забавный',
 'каво',
 'клешня',
 'кой',
 'кофейня',
 'мастерица',
 'мега',
 'молочко',
 'мужесвеннен',
 'насчет',
 'ноль',
 'обставлять',
 'отвратный',
 'переносить',
 'повседневный',
 'повыдергать',
 'привязанность',
 'пчела',
 'расположение',
 'соседский',
 'съезжать',
 'твердо',
 'фаш',
 'фильмец',
 'фрэнсис',
 'халка',
 'червь']

Какой сезон был самым популярным у Чендлера? у Моники?

Чендлер

In [281]:
def popular(name):

    row = data.loc[name.lower()]
    row = row.drop('freq')

    maximum = 0
    seas = ''

    for i in range(1, 8):
        a = row[seasons[i]].sum()
        if maximum < a:
            maximum = a
            seas = i

    return seas

In [282]:
popular('чендлер')

2

Моника

In [283]:
popular('моника')

2

Кто из главных героев статистически самый популярный?

In [284]:
data.loc[['моника', 'росс', 'джоуи', 'чендлер', 'фиби', 'рэйчел']]['freq']

Term
моника     3.881715
росс       5.182460
джоуи      3.999995
чендлер    4.085088
фиби       3.236180
рэйчел     2.427932
Name: freq, dtype: float64

Росс самый популярный

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

Таблица с кол-вом употреблений

In [300]:
new_vect = CountVectorizer()
X = new_vect.fit_transform(list(all_.values())) 
new_data = pd.DataFrame(X.todense()).transpose()
new_data.columns = list(idx.values())
new_data['Term'] = new_vect.get_feature_names()
new_data = new_data.set_index(['Term'])

In [310]:
### реализуйте эту функцию ранжирования
from math import log


def score_BM25(qf, dl, avgdl, k1, b, N, n):
    """
    Compute similarity score between search query and documents from collection
    :return: score

    qf - кол - во вхождений слова в документе
    dl - длина документа

    """

    tf = qf / dl
    idf = log(N - n + 0.5 / n + 0.5)
    a = (k1 + 1) * tf
    b = tf + k1*(1 - b + b*(len(words)) / avgdl)

    return (a / b) * idf

In [311]:
score_BM25(45, 566, avgdl, k1, b, N, 3)

2.0942846416971577

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [312]:
def compute_sim(text, doc, data):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """

    if not isinstance(text, str):
        raise ValueError('enter correct data')
    if not isinstance(doc, str):
        raise ValueError('enter correct data')
    if doc not in data.columns:
        raise ValueError('enter correct data')

    opr = [' ', '  ', '\t', '\n']
    k1 = 2.0
    b = 0.75
    N = 165
    avgdl = np.mean(avr)

    words = mystem.lemmatize(remove(text))
    words = [word for word in words if word not in opr]

    ans = 0
    text_len = data.sum()

    for word in words:
        if word in data.index:
            qf = data.loc[word][doc]
            dl = text_len[doc]
            n = inv_ind.loc[word]['lenght']
            ans += score_BM25(qf, dl, avgdl, k1, b, N, n)

    return ans


def get_search_result(text, data):
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """

    if not isinstance(text, str):
        raise ValueError

    arr = []

    for doc in data.columns:
        arr.append((doc, compute_sim(text, doc, data)))

    return arr

In [317]:
sorted(get_search_result('рождественские каникулы', new_data),
       key=lambda x: x[1], reverse=True)[:10]

[("Friends - 2x09 - The One With Phoebe's Dad", 0.1442948960004289),
 ('Friends - 5x24-25 - The One In Vegas (2)', 0.09786580203636414),
 ('Friends - 3x21 - The One With A Chick And A Duck', 0.08329432985607205),
 ('Friends - 4x19 - The One With All The Haste', 0.05212327690188558),
 ('Friends - 5x15 - The One With The Girl Who Hits Joey', 0.03738228535181276),
 ("Friends - 4x18 - The One With Rachel's New Dress", 0.03729038035910373),
 ("Friends - 5x12 - The One With Chandler's Work Laugh", 0.035376934681449444),
 ("Friends - 5x19 - The One Where Ross Can't Flirt", 0.034629817744540325),
 ('Friends - 1x17 - The One With Two Parts (2)', 0.017897165013359913),
 ('Friends - 3x08 - The One With The Giant Poking Device',
  0.016843805154432533)]